# Dubai Real Estate Database - SQL Magic Quick Start

Welcome to the Dubai Real Estate Database! This notebook will get you up and running with **13+ million real estate records** from Dubai Land Department in just 15 minutes.

## 🎯 What You'll Learn
- Set up ClickHouse connection (Cloud or local)
- Install the complete Dubai real estate database
- Query 1.5M+ transactions with beautiful SQL Magic
- Analyze Dubai's real estate trends (2002-2025)
- Export results for further analysis

## 📊 Datasets Overview
- **1.5M+ transactions** (dld_transactions) - Sales, transfers, mortgages
- **2.2M+ property units** (dld_units) - Apartments, villas, offices
- **8.9M+ rental contracts** (dld_rent_contracts) - Lease agreements
- **223K+ buildings** (dld_buildings) - Building specifications
- **Plus 14 more datasets** with licenses, permits, valuations, and more!

## 🚀 Step 1: Installation

First, make sure you have the package installed:

In [ ]:
# Install the package (run this if not already installed)
# !pip install dubai_real_estate

# Import required modules
from dubai_real_estate.connection import create_connection
from dubai_real_estate.install import install_database
import time
from datetime import datetime

print("✅ Dubai Real Estate package imported successfully!")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")

## 🔗 Step 2: Set Up ClickHouse Connection

You need a ClickHouse instance. Choose one of these options:

### Option A: ClickHouse Cloud (Recommended)
Get a **[1-month free trial](https://clickhouse.com/cloud)** - no setup required!

### Option B: Local ClickHouse
Run locally with Docker: `docker run -d -p 8123:8123 clickhouse/clickhouse-server`

In [ ]:
# Option A: ClickHouse Cloud (recommended)
create_connection(
    name="dubai_cloud",
    connection_type="client",
    host="your-instance.clickhouse.cloud",  # Replace with your ClickHouse Cloud host
    port=8443,
    username="default",  # Usually 'default'
    password="your-password",  # Replace with your password
    secure=True,
    set_auto=True,  # This makes it the default connection
    description="ClickHouse Cloud instance for Dubai real estate data"
)

# Option B: Local ClickHouse (uncomment if using local setup)
# create_connection(
#     name="dubai_local",
#     connection_type="client",
#     host="localhost",
#     port=8123,
#     username="default",
#     password="",
#     secure=False,
#     set_auto=True,
#     description="Local ClickHouse for development"
# )

print("✅ ClickHouse connection created successfully!")
print("🔗 Connection will be used automatically for all operations")

## 📥 Step 3: Install Dubai Real Estate Database

This will download and install **13+ million records** of Dubai real estate data. 

**⏱️ Expected time: ~14 minutes on ClickHouse Cloud**

Grab a coffee ☕ while the data loads!

In [ ]:
# Time the installation
start_time = time.time()
print(f"🚀 Starting installation at {datetime.now().strftime('%H:%M:%S')}")
print("📊 Installing 18 datasets with 13+ million records...")
print("⏳ This will take approximately 14 minutes...")
print()

# Install the complete database
result = install_database(
    database_name="dubai_real_estate",
    include_functions=True,     # Install helper SQL functions
    include_tables=True,       # Install staging tables
    include_views=True,        # Install clean data views
    include_prod_tables=True,  # Install production tables
    cleanup_after_prod=True    # Clean up staging after production
)

# Calculate duration
duration_minutes = (time.time() - start_time) / 60

print(f"\n🏁 Installation completed in {duration_minutes:.2f} minutes!")
print(f"✅ Success: {result['success']}")
print(f"📊 Database: {result['database_name']}")

if result.get('validation'):
    validation = result['validation']
    print(f"📈 Total records processed: {validation.get('total_prod_rows', 0):,}")
    print(f"📋 Tables with data: {validation.get('tables_with_data', 0)}/{validation.get('tables_checked', 0)}")

if result.get('total_errors', 0) > 0:
    print(f"⚠️  Errors encountered: {result['total_errors']}")
else:
    print("🎉 Perfect installation - no errors!")

## 🎨 Step 4: Load SQL Magic Extension

Now for the fun part! Load the beautiful SQL Magic extension that makes querying a joy.

In [ ]:
# Load the SQL Magic extension
%load_ext dubai_real_estate.sql

print("🎨 SQL Magic loaded successfully!")
print("📝 You can now use %sql and %%sql commands")
print("🎯 Try: %sql SELECT 1 as test")

## 📊 Step 5: Your First Queries

Let's explore the data! Start with some simple queries to see what we have.

In [ ]:
# Check total transactions
%sql SELECT COUNT(*) as total_transactions FROM dld_transactions

In [ ]:
# Check all available tables
%sql_tables

In [ ]:
# Quick overview of all datasets
%%sql
SELECT 
    'dld_transactions' as dataset,
    COUNT(*) as records,
    'Real estate transactions' as description
FROM dld_transactions

UNION ALL

SELECT 
    'dld_units' as dataset,
    COUNT(*) as records,
    'Property units' as description
FROM dld_units

UNION ALL

SELECT 
    'dld_rent_contracts' as dataset,
    COUNT(*) as records,
    'Rental contracts' as description
FROM dld_rent_contracts

UNION ALL

SELECT 
    'dld_buildings' as dataset,
    COUNT(*) as records,
    'Buildings' as description
FROM dld_buildings

ORDER BY records DESC;

## 📈 Step 6: Analyze Dubai Real Estate Trends

Now let's dive into some real analysis! Explore Dubai's real estate market trends over the years.

In [ ]:
# Transaction trends by year (2020-2025)
%%sql
SELECT 
    year(instance_date) as year,
    COUNT(*) as total_transactions,
    COUNT(CASE WHEN actual_worth > 0 THEN 1 END) as paid_transactions,
    round(AVG(actual_worth), 0) as avg_amount_aed,
    round(SUM(actual_worth) / 1000000, 1) as total_value_millions_aed
FROM dld_transactions 
WHERE instance_date >= '2020-01-01' 
    AND instance_date < '2026-01-01'
    AND actual_worth > 0
GROUP BY year
ORDER BY year DESC;

In [ ]:
# Top 15 areas by transaction volume
%%sql
SELECT 
    area_name_english as area,
    COUNT(*) as total_transactions,
    COUNT(CASE WHEN actual_worth > 0 THEN 1 END) as paid_transactions,
    round(AVG(CASE WHEN actual_worth > 0 THEN actual_worth END), 0) as avg_price_aed,
    round(MAX(actual_worth), 0) as max_price_aed
FROM dld_transactions 
WHERE area_name_english IS NOT NULL
    AND instance_date >= '2020-01-01'
GROUP BY area_name_english 
HAVING paid_transactions >= 100
ORDER BY total_transactions DESC 
LIMIT 15;

In [ ]:
# Property types analysis
%%sql
SELECT 
    property_type_english as property_type,
    COUNT(*) as transactions,
    round(AVG(CASE WHEN actual_worth > 0 THEN actual_worth END), 0) as avg_price_aed,
    round(MIN(CASE WHEN actual_worth > 0 THEN actual_worth END), 0) as min_price_aed,
    round(MAX(actual_worth), 0) as max_price_aed
FROM dld_transactions 
WHERE property_type_english IS NOT NULL
    AND actual_worth > 0
    AND instance_date >= '2022-01-01'
GROUP BY property_type_english 
HAVING transactions >= 1000
ORDER BY avg_price_aed DESC;

## 🏘️ Step 7: Explore Specific Areas

Let's look at some of Dubai's most famous areas in detail.

In [ ]:
# Downtown Dubai analysis
%%sql
SELECT 
    year(instance_date) as year,
    property_type_english as property_type,
    COUNT(*) as transactions,
    round(AVG(actual_worth), 0) as avg_price_aed,
    round(MAX(actual_worth), 0) as max_price_aed
FROM dld_transactions 
WHERE area_name_english = 'Downtown Dubai'
    AND actual_worth > 0
    AND instance_date >= '2020-01-01'
    AND property_type_english IS NOT NULL
GROUP BY year, property_type_english
HAVING transactions >= 10
ORDER BY year DESC, avg_price_aed DESC;

In [ ]:
# Palm Jumeirah luxury market
%%sql
SELECT 
    property_type_english as property_type,
    property_sub_type_english as sub_type,
    COUNT(*) as transactions,
    round(AVG(actual_worth), 0) as avg_price_aed,
    round(MIN(actual_worth), 0) as min_price_aed,
    round(MAX(actual_worth), 0) as max_price_aed
FROM dld_transactions 
WHERE area_name_english = 'Palm Jumeirah'
    AND actual_worth > 0
    AND instance_date >= '2022-01-01'
    AND property_type_english IS NOT NULL
GROUP BY property_type_english, property_sub_type_english
HAVING transactions >= 5
ORDER BY avg_price_aed DESC;

## 🏢 Step 8: Explore Buildings and Units

Let's look at the property inventory data.

In [ ]:
# Building analysis by area
%%sql
SELECT 
    area_name_english as area,
    COUNT(DISTINCT building_name) as unique_buildings,
    COUNT(*) as total_units,
    round(AVG(property_size_sqft), 1) as avg_size_sqft
FROM dld_units 
WHERE area_name_english IS NOT NULL
    AND property_size_sqft > 0
GROUP BY area_name_english 
HAVING unique_buildings >= 50
ORDER BY total_units DESC 
LIMIT 15;

In [ ]:
# Property sizes by type
%%sql
SELECT 
    property_type_english as property_type,
    property_sub_type_english as sub_type,
    COUNT(*) as units,
    round(AVG(property_size_sqft), 1) as avg_size_sqft,
    round(MIN(property_size_sqft), 1) as min_size_sqft,
    round(MAX(property_size_sqft), 1) as max_size_sqft
FROM dld_units 
WHERE property_type_english IS NOT NULL
    AND property_sub_type_english IS NOT NULL
    AND property_size_sqft > 0
    AND property_size_sqft < 50000  -- Filter out outliers
GROUP BY property_type_english, property_sub_type_english
HAVING units >= 1000
ORDER BY avg_size_sqft DESC;

## 💼 Step 9: Real Estate Market Professionals

Explore the real estate services industry.

In [ ]:
# Real estate offices by area
%%sql
SELECT 
    area_name_english as area,
    COUNT(*) as offices,
    COUNT(CASE WHEN office_status_english = 'Active' THEN 1 END) as active_offices
FROM dld_offices 
WHERE area_name_english IS NOT NULL
GROUP BY area_name_english 
HAVING offices >= 10
ORDER BY active_offices DESC
LIMIT 10;

In [ ]:
# Active brokers overview
%%sql
SELECT 
    COUNT(*) as total_brokers,
    COUNT(CASE WHEN broker_status_english = 'Active' THEN 1 END) as active_brokers,
    COUNT(DISTINCT nationality_english) as nationalities,
    COUNT(CASE WHEN gender_english = 'Male' THEN 1 END) as male_brokers,
    COUNT(CASE WHEN gender_english = 'Female' THEN 1 END) as female_brokers
FROM dld_brokers;

## 📤 Step 10: Export Data for Analysis

Use SQL Magic's export feature to save results for further analysis.

In [ ]:
# Export Downtown Dubai transactions to CSV
%sql --export csv SELECT area_name_english, property_type_english, instance_date, actual_worth FROM dld_transactions WHERE area_name_english = 'Downtown Dubai' AND actual_worth > 0 LIMIT 1000

In [ ]:
# Export yearly summary for visualization
%%sql --export csv
SELECT 
    year(instance_date) as year,
    COUNT(*) as transactions,
    round(AVG(actual_worth), 0) as avg_price,
    round(SUM(actual_worth) / 1000000, 1) as total_value_millions
FROM dld_transactions 
WHERE actual_worth > 0 
    AND instance_date >= '2010-01-01'
GROUP BY year
ORDER BY year;

## ⚙️ Step 11: SQL Magic Configuration

Customize the display and behavior of SQL Magic.

In [ ]:
# View current configuration
%sql_config

In [ ]:
# Customize display settings
%sql_config max_rows_display=20
%sql_config show_execution_time=True
%sql_config show_row_count=True

In [ ]:
# Use minimal mode for cleaner output
%sql --minimal SELECT COUNT(*) as total FROM dld_transactions

In [ ]:
# View query history
%sql_history 5

## 🎉 Congratulations!

You've successfully:

✅ **Set up ClickHouse** connection  
✅ **Installed 13+ million records** of Dubai real estate data  
✅ **Explored transactions, units, and buildings** with SQL Magic  
✅ **Analyzed market trends** across Dubai's areas  
✅ **Exported data** for further analysis  

## 🚀 What's Next?

### 📊 Data Analysis Ideas
- **Price prediction models** using historical trends
- **Area comparison dashboards** with property metrics
- **Investment opportunity analysis** by ROI and growth
- **Market timing analysis** for buyers and sellers

### 🔗 Integration Options
- **Business Intelligence**: Connect to Tableau, Power BI, or Grafana
- **Python analysis**: Use pandas with exported CSV data
- **APIs**: Build REST APIs on top of the ClickHouse data
- **Real-time dashboards**: Stream live updates from Dubai Pulse

### 📚 Additional Resources
- **[ClickHouse Documentation](https://clickhouse.com/docs)** - Learn advanced SQL features
- **[Dubai Pulse](https://www.dubaipulse.gov.ae/organisation/dld)** - Official data source
- **[Project GitHub](https://github.com/oualib/dubai_real_estate)** - Contribute and get support

## 📜 Data Attribution

**Data provided by Dubai Land Department via Dubai Pulse**  
Licensed under [Dubai Open Data License](https://www.dubaipulse.gov.ae/docs/DDE%20_%20DRAFT_Open_Data%20Licence_LONG_Form_English%203.pdf)

---

**Happy analyzing! 🏙️📊**